In [14]:
import pandas as pd

# Input Excel file and sheet name
input_excel = "question.xlsx"  # Replace with your Excel file name
sheet_name = "Eng"  # Name of the sheet to read from
query_column = "Neo4j correct query"  # Column name containing the SQL queries

# Output: List of SQL queries
queries = []

def read_queries_from_excel(excel_file, sheet_name, query_column):
    try:
        # Read the Excel file
        df = pd.read_excel(excel_file, sheet_name=sheet_name)

        # Extract the 'Mysql correct query' column
        if query_column in df.columns:
            queries.extend(df[query_column].dropna().tolist())  # Drop any NaN values and convert to list
        else:
            print(f"Column '{query_column}' not found in the sheet.")
        print("Queries loaded successfully.")
        return queries
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Read the queries from the Excel file
queries = read_queries_from_excel(input_excel, sheet_name, query_column)

# Print the generated queries
for i, query in enumerate(queries, start=1):
    print(f"Query {i}: {query}")


Queries loaded successfully.
Query 1: MATCH (adv:Company {symbol: 'ADVANC'})-[:HAS_METRIC]->(adv_metric:Metric {type: 'TotalAssets', year: '2019', quarter: '1'})
RETURN adv_metric.value AS ADVANC_TotalAssets
Query 2: MATCH (aot:Company {symbol: 'AOT'})-[:HAS_METRIC]->(aot_metric:Metric {type: 'TotalLiabilities', year: '2019', quarter: '1'})
RETURN aot_metric.value AS AOT_TotalLiabilities
Query 3: MATCH (bbl:Company {symbol: 'BBL'})-[:HAS_METRIC]->(bbl_metric:Metric {type: 'NetProfitQuarter', year: '2019', quarter: '1'})
RETURN bbl_metric.value AS BBL_NetProfit
Query 4: MATCH (bcp:Company {symbol: 'BCP'})-[:HAS_RATIO]->(bcp_ratio:Ratio {type: 'ROE', year: '2019', quarter: '1'})
RETURN bcp_ratio.value AS BCP_ROE
Query 5: MATCH (bdms:Company {symbol: 'BDMS'})-[:HAS_RATIO]->(bdms_ratio:Ratio {type: 'DE', year: '2019', quarter: '1'})
RETURN bdms_ratio.value AS BDMS_DE_Ratio
Query 6: MATCH (bem:Company {symbol: 'BEM'})-[:HAS_METRIC]->(bem_metric:Metric {type: 'TotalRevenueQuarter', year: '20

In [15]:
queries

["MATCH (adv:Company {symbol: 'ADVANC'})-[:HAS_METRIC]->(adv_metric:Metric {type: 'TotalAssets', year: '2019', quarter: '1'})\nRETURN adv_metric.value AS ADVANC_TotalAssets",
 "MATCH (aot:Company {symbol: 'AOT'})-[:HAS_METRIC]->(aot_metric:Metric {type: 'TotalLiabilities', year: '2019', quarter: '1'})\nRETURN aot_metric.value AS AOT_TotalLiabilities",
 "MATCH (bbl:Company {symbol: 'BBL'})-[:HAS_METRIC]->(bbl_metric:Metric {type: 'NetProfitQuarter', year: '2019', quarter: '1'})\nRETURN bbl_metric.value AS BBL_NetProfit",
 "MATCH (bcp:Company {symbol: 'BCP'})-[:HAS_RATIO]->(bcp_ratio:Ratio {type: 'ROE', year: '2019', quarter: '1'})\nRETURN bcp_ratio.value AS BCP_ROE",
 "MATCH (bdms:Company {symbol: 'BDMS'})-[:HAS_RATIO]->(bdms_ratio:Ratio {type: 'DE', year: '2019', quarter: '1'})\nRETURN bdms_ratio.value AS BDMS_DE_Ratio",
 "MATCH (bem:Company {symbol: 'BEM'})-[:HAS_METRIC]->(bem_metric:Metric {type: 'TotalRevenueQuarter', year: '2019', quarter: '1'})\nRETURN bem_metric.value AS BEM_Tota

In [16]:
len(queries)

100

In [17]:
import time
import psutil
import os
import json
import csv
from decimal import Decimal
from datetime import date, datetime
from neo4j import GraphDatabase

# Neo4j connection setup
uri = "bolt://localhost:7687"
user = "neo4j"
password = "Neo4jpassword"

driver = GraphDatabase.driver(uri, auth=(user, password))
session = driver.session()

# Output file
output_csv = "query_results_with_metrics_neo4j.csv"
repetitions = 10  # Number of times to execute each query for averaging

# Function to monitor RAM usage
def get_ram_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 ** 2)  # Convert bytes to MB

# Function to convert result to JSON-serializable format
def convert_to_serializable(data):
    if isinstance(data, dict):
        return {k: (float(v) if isinstance(v, Decimal) else str(v) if isinstance(v, (date, datetime)) else v) 
                for k, v in data.items()}
    # Handle cases where the result is a list or other data structure
    elif isinstance(data, (list, tuple)):
        return [convert_to_serializable(item) for item in data]
    return data

# Function to measure query execution time with averaging
def measure_query_time(query, repetitions):
    total_time = 0
    for _ in range(repetitions):
        start_time = time.time()
        session.run(query)  # Execute the query
        end_time = time.time()
        total_time += (end_time - start_time)
    return total_time / repetitions

# Function to run queries and save results
def run_queries_and_export_with_metrics(queries, output_csv):
    try:
        # List to store results
        all_results = []

        for index, query in enumerate(queries):
            # Measure RAM before execution
            start_ram = get_ram_usage()

            # Measure execution time with averaging
            avg_execution_time = measure_query_time(query, repetitions)

            # Get the actual query result (run once)
            result = session.run(query).single()  # Use .single() to get a single record

            # Measure RAM after execution
            end_ram = get_ram_usage()

            # Calculate RAM usage
            ram_usage = end_ram - start_ram

            # Prepare row data
            row = {
                "query_index": index + 1,
                "query": query,
                "result": json.dumps(convert_to_serializable(result.data())) if result else "NULL",  # Convert to JSON serializable
                "time": avg_execution_time,  # Average time in seconds
                "ram": ram_usage  # RAM usage in MB
            }
            all_results.append(row)

        # Write results to CSV
        if all_results:
            with open(output_csv, mode="w", encoding="utf-8-sig", newline="") as file:
                writer = csv.DictWriter(file, fieldnames=all_results[0].keys())
                writer.writeheader()
                writer.writerows(all_results)

        print(f"Results with metrics exported to {output_csv}")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        session.close()
        driver.close()

# Execute the function
run_queries_and_export_with_metrics(queries, output_csv)


C:\Users\china\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\neo4j\_sync\work\result.py:595: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


Results with metrics exported to query_results_with_metrics_neo4j.csv
